In [0]:
import pandas as pd
import requests
from pandas.io.json import json_normalize

In [0]:
# Grab NYC Childhood Blood Lead Testing and Elevated Incidence by Zip Code Data
url1 = 'https://health.data.ny.gov/resource/rzpj-maap.json'
url2 = 'https://data.cityofnewyork.us/resource/kku6-nxdu.json'
r1 = requests.get(url1)
r2 = requests.get(url2)
df1 = json_normalize(r1.json())
df2 = pd.DataFrame(r2.json()[1:], columns=r2.json()[0])
test = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1V-yf8UW3Ui189qzhiT4t8MYNEkNHtd9W8S_sN95N4Lo' +
                   '/export?gid=1060511533&format=csv'
                  )
# df = pd.read_csv(BytesIO(data), index_col=0)
# print(df2.jurisdiction_name.head())

In [0]:
#Let's see what our NYC Childhood Blood Lead data looks like: 
print(len(df1.index))
df1.head(3)

1000


,_10to15,_15,_5_10_mcg_dl,county,county_location.coordinates,county_location.type,fips,less_than_5_mcg_dl,percent,rate_per_1_000,tests,total_eblls,year,zip,zip_code_location.coordinates,zip_code_location.type
0,NaN,NaN,NaN,Albany,"[-73.9740136, 42.5882713]",Point,1,NaN,NaN,NaN,NaN,NaN,2000,11220,"[-74.01819492, 40.64057676]",Point
1,NaN,NaN,NaN,Albany,"[-73.9740136, 42.5882713]",Point,1,NaN,NaN,NaN,NaN,NaN,2001,11220,"[-74.01819492, 40.64057676]",Point
2,NaN,NaN,NaN,Albany,"[-73.9740136, 42.5882713]",Point,1,NaN,NaN,NaN,NaN,NaN,2005,11510,"[-73.60464708, 40.65343524]",Point


In [0]:
# Lets peep into our NYC Zipcode Data
print(df2.columns)
df2.head(3)

,count_american_indian,count_asian_non_hispanic,count_black_non_hispanic,count_citizen_status_total,count_citizen_status_unknown,count_ethnicity_total,count_ethnicity_unknown,count_female,count_gender_total,count_gender_unknown,...,percent_nreceives_public_assistance,percent_other_citizen_status,percent_other_ethnicity,percent_pacific_islander,percent_permanent_resident_alien,percent_public_assistance_total,percent_public_assistance_unknown,percent_receives_public_assistance,percent_us_citizen,percent_white_non_hispanic
0,0,28,0,35,0,35,0,19,35,0,...,0.94,0,0,0,0.06,100,0,0.06,0.94,0.17
1,0,1,0,1,0,1,0,1,1,0,...,1,0,0,0,0,100,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
test.head(3)

,State,County,Population,Number of children tested,Percentage of children with levels higher than 5 ug/dl,Percentage of children with levels higher than 10 ug/dl,Percentage of population under the poverty line
0,AL,Autauga County,55136,29,Unknown,0,12.8
1,AL,Baldwin County,191205,357,1.70%,0.6,13.8
2,AL,Barbour County,27119,23,Unknown,4.3,24.1


According to [Medline Plus](https://medlineplus.gov/ency/article/003360.htm), an abnormal level of blood level for children is a blood lead level of 5 µg/dL or greater. Anything greater than this requires further testing and monitoring and the source of lead must be found and removed. In the dataset, we are most interested in the levels of lead above Blood lead level of 5 µg/dL. 
Let's rename some columns and drop columns and rows that are not useful to us.

In [0]:
# Clean Childhood Blood Lead Levels Data

'''Drop rows that do not have any measure of blood lead levels'''
df1.drop(
    df1[
        (df1["less_than_5_mcg_dl"].isnull()) &
        (df1["_5_10_mcg_dl"].isnull()) &
        (df1["_10to15"].isnull()) & 
        (df1["_15"].isnull())
    ].index, 
    inplace=True
)
'''Drop columns that do not add value to what we are measuring (coordinates)'''
df1.drop(
    columns=["county_location.coordinates",
        "county_location.type","zip_code_location.coordinates",
        "zip_code_location.type"], 
    inplace=True
)
'''Rename columns for readability'''
df1 = df1.rename(
    columns={
        "_10to15": "10to15mcg_dl",
        "_5_10_mcg_dl":"5to10mcg_dl",
        "_15": "greater_than_15_mcg_dl"
    })
df1.index = range(len(df1.index))
df1

,10to15mcg_dl,greater_than_15_mcg_dl,5to10mcg_dl,county,fips,less_than_5_mcg_dl,percent,rate_per_1_000,tests,total_eblls,year,zip
0,NaN,NaN,9,Albany,1,38,2.08,20.8,48,NaN,2000,12143
1,NaN,NaN,6,Albany,1,31,NaN,NaN,37,NaN,2001,12143
2,NaN,NaN,6,Albany,1,16,4.35,43.5,23,NaN,2004,12143
3,NaN,NaN,7,Albany,1,43,NaN,NaN,50,NaN,2005,12143
4,NaN,NaN,6,Albany,1,35,2.38,23.8,42,NaN,2006,12143
5,NaN,NaN,7,Albany,1,21,NaN,NaN,28,NaN,2007,12193
6,NaN,NaN,9,Albany,1,33,NaN,NaN,42,NaN,2002,12158
7,NaN,NaN,10,Albany,1,46,1.75,17.5,57,NaN,2004,12158
8,NaN,NaN,10,Albany,1,51,NaN,NaN,61,NaN,2005,12158
9,NaN,NaN,14,Albany,1,66,2.44,24.4,82,NaN,2006,12158


In [0]:
df1['county'].value_counts()

Albany      195
Allegany     28
Name: county, dtype: int64

In [0]:
from vega_datasets import data
imUsingColab = True

if imUsingColab:
    !pip install altair

try:
    import altair as alt
    if imUsingColab:
        alt.renderers.enable('colab')
    else:
        alt.renderers.enable('notebook')    
    imUsingAltair = True
    print('Altair successfully loaded.')

except ModuleNotFoundError:
    imUsingAltair = False
    print('Altair loading failed. Will default to matplotlib.')

Altair successfully loaded.


In [0]:
def map_data(census_df, variable):
    
    '''
    This function creates an interactive cloropleth map using a census variable. 
    '''    
    
    val = pd.read_csv('https://grantmlong.com/data/census_validation.csv', names=['id'])
    census_df['id'] = val
#     cf = val.merge(
#         census_df.loc[
#             census_df[variable]>-1000000,
#             [variable, 'County']],
#         left_on='id',
#         right_on='GEOID',
#         how='left'
#     )
    cf = census_df
    print(cf.head(3))
#     print(cf.County.head(3))
    counties = alt.topo_feature(
        'https://vega.github.io/vega-datasets/data/us-10m.json', 
        'counties'
    )

    census = alt.LookupData(cf, 'id', [variable, 'County'])

    census_map = alt.Chart(
        counties
    ).mark_geoshape(
    ).encode(
        color='{variable}:Q'.format(variable=variable),
        tooltip=[
            '{variable}:Q'.format(variable=variable),
            'County Name:N',
        ]
    ).transform_lookup(
        lookup='id',
        from_= census
    ).project(
        type='albersUsa'
    ).properties(
        width=850,
        height=500
    )
    
    return census_map


In [0]:
map_data(test, 'Percentage of children with levels higher than 10 ug/dl')

  State          County  Population Number of children tested  \
0    AL  Autauga County       55136                        29   
1    AL  Baldwin County      191205                       357   
2    AL  Barbour County       27119                        23   

  Percentage of children with levels higher than 5 ug/dl  \
0                                            Unknown       
1                                              1.70%       
2                                            Unknown       

  Percentage of children with levels higher than 10 ug/dl  \
0                                                  0        
1                                                0.6        
2                                                4.3        

   Percentage of population under the poverty line    id  
0                                             12.8  1001  
1                                             13.8  1003  
2                                             24.1  1005  


Chart({
  data: UrlData({
    format: TopoDataFormat({
      feature: 'counties',
      type: 'topojson'
    }),
    url: 'https://vega.github.io/vega-datasets/data/us-10m.json'
  }),
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'Percentage of children with levels higher than 10 ug/dl:Q'
    }),
    tooltip: [Tooltip({
      shorthand: 'Percentage of children with levels higher than 10 ug/dl:Q'
    }), Tooltip({
      shorthand: 'County Name:N'
    })]
  }),
  height: 500,
  mark: 'geoshape',
  projection: Projection({
    type: 'albersUsa'
  }),
  transform: [LookupTransform({
    from: LookupData({
      data:      State              County  Population Number of children tested  \
      0       AL      Autauga County       55136                        29   
      1       AL      Baldwin County      191205                       357   
      2       AL      Barbour County       27119                        23   
      3       AL         Bibb County       22653                        41   
      4       AL       Blount County       57645                       250   
      5       AL      Bullock County       10693                       148   
      6       AL       Butler County       20523                       102   
      7       AL      Calhoun County      117186                       269   
      8       AL     Chambers County       34091                       127   
      9       AL     Cherokee County       26042                        47   
      10      AL      Chilton County       43781                       304   
      11      AL      Choctaw County       13546                        15   
      12      AL       Clarke County       25331                       133   
      13      AL         Clay County       13617                        63   
      14      AL     Cleburne County       14990                        47   
      15      AL       Coffee County       50726                       152   
      16      AL      Colbert County       54491                       297   
      17      AL      Conecuh County       12985                        34   
      18      AL        Coosa County       11247                         7   
      19      AL    Covington County       37881                       321   
      20      AL     Crenshaw County       13938                        77   
      21      AL      Cullman County       80668                       440   
      22      AL         Dale County       50013                        78   
      23      AL       Dallas County       42743                        19   
      24      AL       DeKalb County       71074                       218   
      25      AL       Elmore County       80321                       100   
      26      AL     Escambia County       38042                        92   
      27      AL       Etowah County      104126                       265   
      28      AL      Fayette County       17019                        31   
      29      AL     Franklin County       31663                       177   
      ...    ...                 ...         ...                       ...   
      1531    WI       Oconto County       37483                       376   
      1532    WI       Oneida County       35754                       339   
      1533    WI    Outagamie County      179139                      1566   
      1534    WI      Ozaukee County       86930                       757   
      1535    WI        Pepin County        7390                        91   
      1536    WI       Pierce County       40859                       410   
      1537    WI         Polk County       43698                       414   
      1538    WI      Portage County       70337                       857   
      1539    WI        Price County       13888                       151   
      1540    WI       Racine County      195039                      3791   
      1541    WI     Richland County       17842                       175   
      154